In [1]:
# import all necessary modules and set parameters
from __future__ import print_function, absolute_import, division
# load modules and cythonmagic
import numpy as np
%load_ext Cython

In [2]:
# Define class Data. It will save all the necessary data for the problem
# Example is about computing interaction between particles with given matrix of interactions
class Data(object):
    """This is container for data for the problem."""
    """First of all, it requires methods check_far, compute_aux, divide and __len__."""
    def __init__(self, particles, matrix):
        """save particles and matrix"""
        self.particles = particles
        self.matrix = matrix
        # main requirement here is to set self.count -- number of particles and self.dim -- dimensionality of the problem
        self.count, self.dim = particles.shape
        
    """All other functions must have exactly the same parameters, as required."""
    
    def check_far(self, bb0, bb1):
        """checks if bounding boxes bb0 and bb1 do not cross each other."""
        mean0 = bb0.mean(axis = 1)
        mean1 = bb1.mean(axis = 1)
        dist = np.linalg.norm(mean1-mean0)
        diag = max(np.linalg.norm(bb0[:,1]-bb0[:,0]), np.linalg.norm(bb1[:,1]-bb1[:,0]))
        return dist > diag
    
    def compute_aux(self, index):
        """computes bounding boxes, requires self.particles defined."""
        selected = self.particles[index]
        return np.hstack([selected.min(axis = 0).reshape(3,1), selected.max(axis = 0).reshape(3,1)])

    def divide(self, index):
        """divides cluster into subclusters."""
        vertex = self.particles[index].copy()
        center = vertex.mean(axis = 0)
        vertex -= center.reshape(1, self.dim)
        normal = np.linalg.svd(vertex, full_matrices = 0)[2][0]
        scal_dot = normal.dot(vertex.T)
        permute = scal_dot.argsort()
        scal_dot = scal_dot[permute]
        k = scal_dot.searchsorted(0)
        return permute, [0, k, permute.size]

    def __len__(self):
        return self.count

In [3]:
%%cython -a
import numpy as np
# this function generates matrix with elements: a[i,...,j] = r/|r|, where r is a radius-vector between points
# in 3d space
from libc.math cimport sqrt

def gen_matrix(x, y):
    a = np.zeros((x.shape[0], 3, y.shape[0]))
    cdef double [:,:,:] a_buf = a
    cdef double [:,:] x_buf = x
    cdef double [:,:] y_buf = y
    cdef double z[3]
    cdef double tmp
    cdef int i, j
    for i in range(x_buf.shape[0]):
        for j in range(y_buf.shape[0]):
            z[0] = x_buf[i,0]-y_buf[j,0]
            z[1] = x_buf[i,1]-y_buf[j,1]
            z[2] = x_buf[i,2]-y_buf[j,2]
            tmp = sqrt(z[0]*z[0]+z[1]*z[1]+z[2]*z[2])
            if tmp > 1e-15:
                a_buf[i, 0, j] = z[0]/tmp
                a_buf[i, 1, j] = z[1]/tmp
                a_buf[i, 2, j] = z[2]/tmp
    return a

In [4]:
# Here we generate 1000 particles randomly in cube [0;1]^3
np.random.seed(100)
particles = np.random.rand(1000, 3)
# Let matrix of interactions be equal to 1/r
dense_matrix = gen_matrix(particles, particles)

In [5]:
# Generate object of class Data based on particles and matrix of interactions
data = Data(particles, dense_matrix)

# Initialize cluster trees with root node
from h2tools import ClusterTree
block_size = 25
tree = ClusterTree(data, block_size)

# Set function, that returns submatrix of the matrix of interactions
def func(data1, rows, data2, columns):
    return data1.matrix[rows][...,columns]

# Hack to prevent bug of qr decomposition for 130x65 matrices (multithread mkl bug)
import os
os.environ['OMP_NUM_THREADS'] = '1'
os.environ['MKL_NUM_THREADS'] = '1'

# Generate block cluster tree in variable `problem`
from h2tools import Problem
symmetric = 0
verbose = True
problem = Problem(func, tree, tree, symmetric, verbose)

Cluster trees are generated in 0.306967020035 seconds
Depth level of each cluster tree: 8
Row cluster tree
    nodes : 121
    leaves: 61
Column cluster tree
    nodes : 121
    leaves: 61


In [6]:
# computing multicharge representation
from __future__ import print_function

from h2tools.mcbh import mcbh

print('Computing MCBH, relative error parameter tau set to 1e-4')
h2matrix = mcbh(problem, tau=1e-4, iters=1, onfly=0, verbose=1)
print('memory for uncompressed approximation: '+str(h2matrix.nbytes()/1024./1024)+' MB')
# error measure with pypropack
# print('relative error of approximation:', h2matrix.diffnorm(far_only=0))
print('cannot measure error with non-scalar function of interaction')
x = np.random.rand(1000)
y = problem.dot(x)
z = h2matrix.dot(x)
print(np.linalg.norm(y-z)/np.linalg.norm(y))
y = problem.rdot(x)
z = h2matrix.rdot(x)
print(np.linalg.norm(y-z)/np.linalg.norm(y))
y = problem.far_dot(x)
z = h2matrix.far_dot(x)
print(np.linalg.norm(y-z)/np.linalg.norm(y))
y = problem.far_rdot(x)
z = h2matrix.far_rdot(x)
print(np.linalg.norm(y-z)/np.linalg.norm(y))

# compressing approximant
h2matrix.svdcompress(1e-3, verbose=1)
y = problem.dot(x)
z = h2matrix.dot(x)
print(np.linalg.norm(y-z)/np.linalg.norm(y))
y = problem.rdot(x)
z = h2matrix.rdot(x)
print(np.linalg.norm(y-z)/np.linalg.norm(y))
y = problem.far_dot(x)
z = h2matrix.far_dot(x)
print(np.linalg.norm(y-z)/np.linalg.norm(y))
y = problem.far_rdot(x)
z = h2matrix.far_rdot(x)
print(np.linalg.norm(y-z)/np.linalg.norm(y))

h2matrix.svdcompress(1e-2, verbose=1)
y = problem.dot(x)
z = h2matrix.dot(x)
print(np.linalg.norm(y-z)/np.linalg.norm(y))
y = problem.rdot(x)
z = h2matrix.rdot(x)
print(np.linalg.norm(y-z)/np.linalg.norm(y))
y = problem.far_dot(x)
z = h2matrix.far_dot(x)
print(np.linalg.norm(y-z)/np.linalg.norm(y))
y = problem.far_rdot(x)
z = h2matrix.far_rdot(x)
print(np.linalg.norm(y-z)/np.linalg.norm(y))

Computing MCBH, relative error parameter tau set to 1e-4
Far-field interactions(MCBH method):
    Function calls: 2164
    Function values computed: 9750627
    Function time, seconds: 1.16
    Average time per function value, seconds: 1.19e-07
    Maxvol time, seconds: 2.07166290283
Near-field interactions:
    Function calls: 1143
    Function values computed: 1061652
    Function time, seconds: 0.08
    Average time per function value, seconds: 7.43e-08
Total time, seconds: 3.38
Memory:
    Basises, MB: 0.04
    Transfer matrices, MB: 0.50
    Far-field interactions, MB: 0.44
    Near-field interactions, MB: 0.32
Total memory, MB: 1.30
memory for uncompressed approximation: 1.30458831787 MB
cannot measure error with non-scalar function of interaction
2.75760415352e-05
2.71649335913e-05
3.53833907199e-05
3.48558895922e-05
memory BEFORE SVD-compression: 1.30MB
memory AFTER SVD-compression: 1.03MB
recompression time: 0.335499048233
0.000172043382905
0.000172663108558
0.000220752432155


In [7]:
print(h2matrix)

H2matrix at 0x105d96c10
    Structure (h2/mcbh): h2
    Memory layout (low/full): full
    Shape: [1000, 3, 1000]
    Total memory, MB: 0.86
        Basises, MB: 0.00
        Transfer matrices, MB: 0.10
        Far-field interactions, MB: 0.44
        Near-field interactions, MB: 0.32
